<h1>Chapter 11. Object in the Spirit of Python</h1>

The "Spirit of Python" encompasses Python's principles of simplicity, readability, versatility, and elegance, defining its philosophy and guiding values. When writing a library or framework, programmers anticipate that the classes they utilize will operate in a manner consistent with Python's own provided classes.

<h2>Object Presentation</h2>

In [1]:
import math
from array import array


class Vector2d:
    # 'd' represents a double-precision floating-point number
    typecode = 'd'

    # Converting x and y to float type in __init__ method
    # allows to detect errors at an early stage
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    # The presence of the __iter__ method makes Vector2d iterable
    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        # Vector2d is an iterated object,
        # *self supplies the x and y components of the format function
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        # Convert the typecode to bytes and concatenate it with the bytes object
        # obtained through array transformation, constructed by traversing the instance
        return bytes([ord(self.typecode)]) + bytes(array(self.typecode, self))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        # The modulus of a vector is the length of the hypotenuse
        # of a right triangle with cathetes x and y
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

In [2]:
v1 = Vector2d(3, 4)

`Vector2d` components can be accessed directly as attributes (no read methods)

In [3]:
print(v1.x, v1.y)

3.0 4.0


A `Vector2d` object can be unpacked into a tuple of variables

In [4]:
x, y = v1

`repr` for a `Vector2d` object mimics the source code for constructing an exemplar

In [5]:
x, y

(3.0, 4.0)

In [6]:
v1

Vector2d(3.0, 4.0)

Using `eval` shows that the result of `repr` for `Vector2d` is an exact representation of the constructor call 

In [7]:
v1_clone = eval(repr(v1))

`Vector2d` supports comparison with `==`; this is useful for testing

In [8]:
v1 == v1_clone

True

`print` calls the `str` function, which generates an ordered pair for `Vector2d`

In [9]:
print(v1)

(3.0, 4.0)


`bytes` uses `__bytes__` method to get binary representation

In [10]:
octets = bytes(v1)
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

`abs` calls the `__abs__` method to return the modulus of the vector

In [11]:
abs(v1)

5.0

`bool` uses the `__bool__` method to return `False` for a `Vector2d` object of zero length, and `True` otherwise

In [12]:
bool(v1), bool(Vector2d(0, 0))

(True, False)

<h2>Alternative Constructor</h2>

Constructing the `Vector2d` class from a binary sequence

In [13]:
import math
from array import array


class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(array(self.typecode, self))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    # Construct Vector2d from a binary sequence
    @classmethod
    def frombytes(cls, octets):  # the class itself is passed to cls instead of self
        typecode = chr(octets[0])  # read typecode from the first byte
        # Create a memoryview object from a binary sequence of octets
        # and cast it to the typecode type
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [14]:
v1 = Vector2d(3, 4)
v1

Vector2d(3.0, 4.0)

In [15]:
v1_clone = Vector2d.frombytes(bytes(v1))
v1_clone

Vector2d(3.0, 4.0)

In [16]:
v1 == v1_clone

True

<h2><code>@classmethod</code> and <code>@staticmethod</code> Decorators</h2>

`@classmethod` creates class methods that receive the class `(cls)` as the first argument. They are used for methods that operate on the class itself or manage class attributes.

`@staticmethod` creates a static method that doesn't receive either the class or the object instance as the first argument. It's essentially a function defined within a class namespace, useful for utility functions not directly related to class or object manipulation.

In [17]:
class Demo:
    @classmethod
    def klassmeth(*args):
        return args  # returns all positional arguments

    @staticmethod
    def statmeth(*args):
        return args  # returns all positional arguments

Regardless of how `Demo.klassmeth` is called, it gets the `Demo` class as its first argument.

In [18]:
Demo.klassmeth()

(__main__.Demo,)

In [19]:
Demo.klassmeth('spam')

(__main__.Demo, 'spam')

`Demo.statmeth` behaves like a normal function

In [20]:
Demo.statmeth()

()

In [21]:
Demo.statmeth('spam')

('spam',)

<h2>Output Formatting</h2>

`format()` and `str.format()` are used for string formatting, allowing you to create formatted strings by replacing placeholders with specified values.

`format(value, format_spec)` is a built-in function used to format a single value according to the specified format. `value` is the data you want to format, and format_spec is a string specifying how the value should be formatted.

In [22]:
# Brazilian real/US dollar exchange rate
brl = 1 / 5.19
brl

0.19267822736030826

In [23]:
format(brl, '0.4f')

'0.1927'

`str.format()` is a method of a string object that replaces placeholders (curly braces `{}`) in the string with values passed as arguments to the method. The placeholders can be numbered or named, allowing you to specify the order in which the values are inserted into the string.

In [24]:
'1 BRL = {rate:0.2f} USD'.format(rate=brl)

'1 BRL = 0.19 USD'

In [25]:
# Convert the integer to a binary string
format(42, 'b')

'101010'

In [26]:
# Percent value output
format(2 / 3, '.1%')

'66.7%'

In [27]:
from datetime import datetime

format(datetime.now(), '%H:%M:%S')

'18:05:19'

In [28]:
'It is now {:%I:%M %p}'.format(datetime.now())

'It is now 06:05 PM'

In [29]:
v1 = Vector2d(3, 4)

try:
    format(v1, '.3f')
except TypeError as e:
    print(e.__repr__())

TypeError('unsupported format string passed to Vector2d.__format__')


Fix by implementing custom mini-formatting language 

In [30]:
import math
from array import array


class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(array(self.typecode, self))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    # The math.atan2() function is used to get the angle
    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        # The format ends with 'p': polar coordinates
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]  # remove suffix 'p'
            coords = (abs(self), self.angle())  # build a tuple of polar coordinates
            outer_fmt = '<{}, {}>'  # build the output format using angle brackets
        else:
            # Use the x, y components of the self vector to represent it
            # in rectangular coordinates
            coords = self
            outer_fmt = '({}, {})'  # build the output format using parentheses
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [31]:
format(Vector2d(1, 1), 'p')

'<1.4142135623730951, 0.7853981633974483>'

In [32]:
format(Vector2d(1, 1), '.3ep')

'<1.414e+00, 7.854e-01>'

In [33]:
format(Vector2d(1, 1), '0.5fp')

'<1.41421, 0.78540>'

<h2>Hashable Class <code>Vector2d</code></h2>

In [34]:
v1 = Vector2d(3, 4)

try:
    hash(v1)
except TypeError as e:
    print(e.__repr__())

TypeError("unhashable type: 'Vector2d'")


In [35]:
try:
    set([v1])
except TypeError as e:
    print(e.__repr__())

TypeError("unhashable type: 'Vector2d'")


Changes needed to make `Vector2d` class immutable

The `@property` decorator allows you to define a method in a class as a property. It enables to access the method like an attribute, providing a cleaner syntax and encapsulation for `getter`, `setter`, and `deleter` of class attributes.

In [36]:
import math
from array import array


class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)  # lock the attribute
        self.__y = float(y)

    # The @property decorator marks the method for reading of a property
    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(array(self.typecode, self))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    # Implement the __hash__ method
    def __hash__(self):
        return hash((self.x, self.y))

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [37]:
v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2)

hash(v1), hash(v2)

(1079245023883434373, 1994163070182233067)

In [38]:
{v1, v2}

{Vector2d(3.0, 4.0), Vector2d(3.1, 4.2)}